In [1]:
import pandas as pd
import numpy as np

In [2]:
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'root',
    password = 'krjniJm6OuIyhEoo',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)
con = engine.connect()

In [3]:
# Query to create a database
db_name = 'bike_sharing'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8mb4'".format(db=db_name)

# Create a database
engine.execute(create_db_query)


/usr/local/lib/python3.5/dist-packages/sqlalchemy/engine/default.py:507: Warning: (1007, "Can't create database 'bike_sharing'; database exists")
  cursor.execute(statement, parameters)


In [4]:
# And lets switch to the database
engine.execute("USE {db}".format(db=db_name))

## Bikeshare Dataset

Dataset description at 
https://www.kaggle.com/contactprad/bike-share-daily-data

- instant: record index
- dteday : date
- season : season (1:springer, 2:summer, 3:fall, 4:winter)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
+ weathersit : 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered

In [5]:
create_table = '''
DROP TABLE hourly;

CREATE TABLE hourly (
  instant smallint(6) COMMENT 'record index',
  dteday date COMMENT 'date',
  season tinyint(4) COMMENT 'season (1:springer, 2:summer, 3:fall, 4:winter)',
  yr tinyint(4) COMMENT 'year (0: 2011, 1:2012)',
  mnth tinyint(4) COMMENT 'month ( 1 to 12)',
  hr tinyint(4) COMMENT 'hour (0 to 23)',
  holiday tinyint(4) COMMENT 'weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)',
  weekday tinyint(4) COMMENT 'day of the week',
  workingday tinyint(4) COMMENT 'if day is neither weekend nor holiday is 1, otherwise is 0.',
  weathersit tinyint(4) COMMENT '- 1: Clear, Few clouds, Partly cloudy, Partly cloudy\n- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist\n- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds\n- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog',
  temp float COMMENT 'Normalized temperature in Celsius. The values are divided to 41 (max)',
  atemp float COMMENT 'Normalized feeling temperature in Celsius. The values are divided to 50 (max)',
  hum float COMMENT 'Normalized humidity. The values are divided to 100 (max)',
  windspeed float COMMENT 'Normalized wind speed. The values are divided to 67 (max)',
  casual smallint(6) COMMENT 'count of casual users',
  registered smallint(6) COMMENT 'count of registered users',
  cnt smallint(6) COMMENT 'count of total rental bikes including both casual and registered'
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

DROP TABLE daily;

CREATE TABLE daily (
  instant smallint(6) COMMENT 'record index',
  dteday date COMMENT 'date',
  season tinyint(4) COMMENT 'season (1:springer, 2:summer, 3:fall, 4:winter)',
  yr tinyint(4) COMMENT 'year (0: 2011, 1:2012)',
  mnth tinyint(4) COMMENT 'month ( 1 to 12)',
  holiday tinyint(4) COMMENT 'weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)',
  weekday tinyint(4) COMMENT 'day of the week',
  workingday tinyint(4) COMMENT 'if day is neither weekend nor holiday is 1, otherwise is 0.',
  weathersit tinyint(4) COMMENT '- 1: Clear, Few clouds, Partly cloudy, Partly cloudy\n- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist\n- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds\n- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog',
  temp float COMMENT 'Normalized temperature in Celsius. The values are divided to 41 (max)',
  atemp float COMMENT 'Normalized feeling temperature in Celsius. The values are divided to 50 (max)',
  hum float COMMENT 'Normalized humidity. The values are divided to 100 (max)',
  windspeed float COMMENT 'Normalized wind speed. The values are divided to 67 (max)',
  casual smallint(6) COMMENT 'count of casual users',
  registered smallint(6) COMMENT 'count of registered users',
  cnt smallint(6) COMMENT 'count of total rental bikes including both casual and registered'
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''

engine.execute(create_table)

### Read Daily Dataset

In [6]:
df = pd.read_csv("bike_sharing_daily.csv", low_memory = False)

In [7]:
df.dteday = pd.to_datetime(df.dteday)

In [8]:
df.dtypes

instant                int64
dteday        datetime64[ns]
season                 int64
yr                     int64
mnth                   int64
holiday                int64
weekday                int64
workingday             int64
weathersit             int64
temp                 float64
atemp                float64
hum                  float64
windspeed            float64
casual                 int64
registered             int64
cnt                    int64
dtype: object

In [9]:
table_name = 'daily'
df.to_sql(
        name = table_name, 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False, 
        chunksize = 1000)

### Read Hourly Dataset

In [10]:
df = pd.read_csv("bike_sharing_hourly.csv", low_memory = False)
df.dteday = pd.to_datetime(df.dteday)

In [11]:
df.dtypes

instant                int64
dteday        datetime64[ns]
season                 int64
yr                     int64
mnth                   int64
hr                     int64
holiday                int64
weekday                int64
workingday             int64
weathersit             int64
temp                 float64
atemp                float64
hum                  float64
windspeed            float64
casual                 int64
registered             int64
cnt                    int64
dtype: object

In [12]:
table_name = 'hourly'
df.to_sql(
        name = table_name, 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False, 
        chunksize = 1000)